In [1]:
# !pip install ydata-profiling pandas

In [3]:
%pip install matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 1.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 31.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 65.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 10.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import logging
import warnings

# отключаем ворнинги
warnings.filterwarnings("ignore")

# отключаем прогресс-бары
os.environ["YDATA_PROFILING_DISABLE_PROGRESS_BAR"] = "1"
os.environ["PROGRESS_BAR_OFF"] = "True"

# отключаем логи библиотек
logging.getLogger("ydata_profiling").setLevel(logging.CRITICAL)
logging.getLogger("pandas_profiling").setLevel(logging.CRITICAL)
logging.getLogger("matplotlib").setLevel(logging.CRITICAL)

%matplotlib inline

In [19]:
import sqlite3
from pathlib import Path

import pandas as pd

try:
    from ydata_profiling import ProfileReport
    HAS_PROFILING = True
except ImportError:
    HAS_PROFILING = False
    print("⚠ ydata_profiling не установлен — профили не будут строиться.")


from pathlib import Path

ROOT = Path.cwd()   # текущая директория ноутбука
DB_PATH = ROOT.parent / "db" / "shop.db"
print("Используем БД:", DB_PATH.resolve())

conn = sqlite3.connect(DB_PATH)


⚠ ydata_profiling не установлен — профили не будут строиться.
Используем БД: /home/recsys1_user01/recipe_recommender/llm_work/recipe_recommender/db/shop.db


In [20]:
def list_tables(connection):
    query = """
    SELECT name
    FROM sqlite_master
    WHERE type='table'
      AND name NOT LIKE 'sqlite_%'
    ORDER BY name
    """
    return pd.read_sql(query, connection)["name"].tolist()

tables = list_tables(conn)
tables


['category',
 'event',
 'order_item',
 'orders',
 'product',
 'recipe',
 'recipe_product',
 'session_info',
 'user']

In [21]:
def load_table(connection, table_name: str) -> pd.DataFrame:
    return pd.read_sql(f"SELECT * FROM {table_name}", connection)

# Пример: посмотрим на product
if "product" in tables:
    df_product = load_table(conn, "product")
    display(df_product.head())


,id,name,image_url,price,unit,weight_value,weight_unit,rating,shelf_life_days,category_id,brand,description,composition,metadata
0,1,"Абрикос мытый, Фрешбар",https://img.vkusvill.ru/pim/images/site/62d3b0...,165.0,шт,300.0,г,0.0,None,0,ВкусВилл,"Медовой спелости абрикосы, тщательно отобранны...",абрикос свежий,"{""category"": ""Фрукты"", ""subcategory"": ""Фруктов..."
1,2,Абрикос Планета Витаминов без косточки заморож...,https://img.vkusvill.ru/pim/images/site/a2549a...,199.0,шт,250.0,г,4.6,None,1,Планета Витаминов,Линейка Гурманики дополняется новым продуктом ...,Абрикос,"{""category"": ""Замороженные ягоды"", ""subcategor..."
2,3,Абрикосы сушеные,https://img.vkusvill.ru/pim/images/site/96ffdd...,328.0,шт,200.0,г,4.8,None,2,ВкусВилл,Сушеные абрикосы способны быстро утолить голод...,абрикосы сушеные без косточки,"{""category"": ""Орехи, чипсы и снеки"", ""subcateg..."
3,5,Аджика,https://img.vkusvill.ru/pim/images/site/site/2...,184.0,шт,200.0,г,4.8,None,3,ВкусВилл,Острая аджика с насыщенным вкусом и густой тек...,"перец красный сладкий свежий, паста томатная (...","{""category"": ""Консервация"", ""subcategory"": ""Ху..."
4,6,Аджика / Соус Фирменный KERAKUR,https://img.vkusvill.ru/pim/images/site/259f77...,335.0,шт,480.0,г,4.8,None,1,Kerakur,"Аджика неострая , подается к мясным и куриным ...","красный перец, томатная паста, красный острый ...","{""category"": ""Овощные консервы"", ""subcategory""..."


In [22]:
reports_dir = Path("table_reports")
reports_dir.mkdir(exist_ok=True)

key_tables = ["user", "product", "orders", "order_item", "event"]

for t in key_tables:
    if t not in tables:
        continue
    print("=" * 60)
    print(f"⭐ Ключевая таблица: {t}")
    print("=" * 60)
    
    df = load_table(conn, t)
    display(df)
    
    if HAS_PROFILING and len(df) > 0:
        profile = ProfileReport(df, title=f"Key table profile: {t}", minimal=True)
        html_path = reports_dir / f"key_{t}_profile.html"
        profile.to_file(html_path)
        print(f"Profile для {t} сохранён в {html_path}\n")


⭐ Ключевая таблица: user


,id,username,password_hash,created_at
0,1,asnp,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-16 23:01:01
1,2,test2,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-16 23:04:13
2,3,test22,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-17 03:27:33
3,4,test123,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-17 03:30:51
4,5,test1234,03ac674216f3e15c761ee1a5e255f067953623c8b388b4...,2025-11-17 03:32:27
5,6,test12345,48875c7264f7e22024b8953a1c8884972d97ff724a84e3...,2025-11-17 05:30:51
6,7,test123456,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-17 05:31:56
7,8,123451,a665a45920422f9d417e4867efdc4fb8a04a1f3fff1fa0...,2025-11-17 07:51:53
8,9,123123123,932f3c1b56257ce8539ac269d7aab42550dacf8818d075...,2025-11-17 07:56:13
9,10,фыв,6ce2b11b62f74576177a63f621be551a946ca1a6f857f4...,2025-11-17 08:43:02


⭐ Ключевая таблица: product


,id,name,image_url,price,unit,weight_value,weight_unit,rating,shelf_life_days,category_id,brand,description,composition,metadata
0,1,"Абрикос мытый, Фрешбар",https://img.vkusvill.ru/pim/images/site/62d3b0...,165.0,шт,300.0,г,0.0,None,0,ВкусВилл,"Медовой спелости абрикосы, тщательно отобранны...",абрикос свежий,"{""category"": ""Фрукты"", ""subcategory"": ""Фруктов..."
1,2,Абрикос Планета Витаминов без косточки заморож...,https://img.vkusvill.ru/pim/images/site/a2549a...,199.0,шт,250.0,г,4.6,None,1,Планета Витаминов,Линейка Гурманики дополняется новым продуктом ...,Абрикос,"{""category"": ""Замороженные ягоды"", ""subcategor..."
2,3,Абрикосы сушеные,https://img.vkusvill.ru/pim/images/site/96ffdd...,328.0,шт,200.0,г,4.8,None,2,ВкусВилл,Сушеные абрикосы способны быстро утолить голод...,абрикосы сушеные без косточки,"{""category"": ""Орехи, чипсы и снеки"", ""subcateg..."
3,5,Аджика,https://img.vkusvill.ru/pim/images/site/site/2...,184.0,шт,200.0,г,4.8,None,3,ВкусВилл,Острая аджика с насыщенным вкусом и густой тек...,"перец красный сладкий свежий, паста томатная (...","{""category"": ""Консервация"", ""subcategory"": ""Ху..."
4,6,Аджика / Соус Фирменный KERAKUR,https://img.vkusvill.ru/pim/images/site/259f77...,335.0,шт,480.0,г,4.8,None,1,Kerakur,"Аджика неострая , подается к мясным и куриным ...","красный перец, томатная паста, красный острый ...","{""category"": ""Овощные консервы"", ""subcategory""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8704,10458,"Зразы ""Сахалинские"" из горбуши с сыром, зам",https://img.vkusvill.ru/pim/images/site/5478ed...,425.0,шт,300.0,г,4.7,None,10,ВкусВилл,"Зразы с горбушей и сыром, которые при обжарке ...","горбуша, сыр полутвердый м.д.ж в сухом вещест...","{""category"": ""Рыба, икра и морепродукты"", ""sub..."
8705,10459,Зубатка пестрая стейк мороженая,https://img.vkusvill.ru/pim/images/site/1a96f9...,920.0,кг,NaN,,4.6,None,10,ВкусВилл,Дикую зубатку вылавливают в Баренцевом море. А...,"Зубатка пестрая, вода питьевая (защитная глазу...","{""category"": ""Рыба, икра и морепродукты"", ""sub..."
8706,10460,Зубатка пестрая стейк зам,https://img.vkusvill.ru/pim/images/site/7ebdd6...,910.0,кг,NaN,,4.9,None,10,ВкусВилл,Замороженные стейки дикой пёстрой зубатки. Мяс...,"зубатка пёстрая, вода питьевая (защитная глазурь)","{""category"": ""Рыба, икра и морепродукты"", ""sub..."
8707,10461,Зубатка пестрая стейк зам,https://img.vkusvill.ru/pim/images/site/9b5aa6...,595.0,шт,600.0,г,4.7,None,10,ВкусВилл,Замороженные стейки дикой пёстрой зубатки. Мяс...,"Зубатка пестрая, вода питьевая (защитная глазу...","{""category"": ""Рыба, икра и морепродукты"", ""sub..."


⭐ Ключевая таблица: orders


,id,user_id,order_time,status,total_price
0,1,1,2025-11-16 23:03:48,created,1591.0
1,2,1,2025-11-16 23:04:05,created,4950.0
2,3,1,2025-11-16 23:39:57,created,835.0
3,4,15,2025-11-17 13:20:27,created,2283.0
4,5,15,2025-11-17 13:22:09,created,1549.0
5,6,15,2025-11-17 13:46:26,created,3032.0
6,7,24,2025-11-17 19:54:37,created,2071.0
7,8,29,2025-11-17 20:02:01,created,1814.0
8,9,34,2025-11-17 20:15:20,created,2356.0


⭐ Ключевая таблица: order_item


,id,order_id,product_id,quantity,price
0,1,1,4324,1,115.0
1,2,1,446,1,239.0
2,3,1,8852,1,420.0
3,4,1,10128,1,209.0
4,5,1,1528,1,608.0
5,6,2,1,30,165.0
6,7,3,1,2,165.0
7,8,3,3,1,328.0
8,9,3,409,1,177.0
9,10,4,8780,1,1722.0


⭐ Ключевая таблица: event


,id,user_id,session_id,event_time,event_type,item_id,page_type,source,experiment_key,variant,position,value,request_id,metadata
0,1,1,0ef7b4abecac4382914ce4dcefde59fd,2025-11-16 23:01:33,add_to_cart,4324.0,search,search,None,None,1.0,None,fdb61fbebd95459aa87aceb4df5e0a1f,"{""query"": ""спагетти"", ""cart"": {""4324"": 1}}"
1,2,1,0ef7b4abecac4382914ce4dcefde59fd,2025-11-16 23:01:33,rec_impression,1528.0,recs_sidebar,recs,None,None,1.0,None,2d5bfea386484bb58cfc5599e7985d68,"{""cart"": {""4324"": 1}}"
2,3,1,0ef7b4abecac4382914ce4dcefde59fd,2025-11-16 23:01:33,rec_impression,4188.0,recs_sidebar,recs,None,None,2.0,None,2d5bfea386484bb58cfc5599e7985d68,"{""cart"": {""4324"": 1}}"
3,4,1,0ef7b4abecac4382914ce4dcefde59fd,2025-11-16 23:01:33,rec_impression,4577.0,recs_sidebar,recs,None,None,3.0,None,2d5bfea386484bb58cfc5599e7985d68,"{""cart"": {""4324"": 1}}"
4,5,1,0ef7b4abecac4382914ce4dcefde59fd,2025-11-16 23:01:33,rec_impression,4589.0,recs_sidebar,recs,None,None,4.0,None,2d5bfea386484bb58cfc5599e7985d68,"{""cart"": {""4324"": 1}}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626,2627,45,3d1c9812f9654025afa0c98c550638b8,2025-11-17 22:00:16,rec_impression,6315.0,recs_sidebar,recs,main_recs_ab,ease_popular,4.0,None,1941662608fd4fa28819dd9936ccc6ca,"{""cart"": {""2"": 1, ""3"": 2, ""5"": 2, ""1040"": 1, ""..."
2627,2628,45,3d1c9812f9654025afa0c98c550638b8,2025-11-17 22:00:16,rec_impression,6332.0,recs_sidebar,recs,main_recs_ab,ease_popular,5.0,None,1941662608fd4fa28819dd9936ccc6ca,"{""cart"": {""2"": 1, ""3"": 2, ""5"": 2, ""1040"": 1, ""..."
2628,2629,45,3d1c9812f9654025afa0c98c550638b8,2025-11-17 22:00:16,rec_impression,8452.0,recs_sidebar,recs,main_recs_ab,ease_popular,6.0,None,1941662608fd4fa28819dd9936ccc6ca,"{""cart"": {""2"": 1, ""3"": 2, ""5"": 2, ""1040"": 1, ""..."
2629,2630,45,3d1c9812f9654025afa0c98c550638b8,2025-11-17 22:00:16,rec_impression,8689.0,recs_sidebar,recs,main_recs_ab,ease_popular,7.0,None,1941662608fd4fa28819dd9936ccc6ca,"{""cart"": {""2"": 1, ""3"": 2, ""5"": 2, ""1040"": 1, ""..."


In [23]:
def max_matching(adj: dict[int, list[int]]) -> dict[int, int]:

    matchR = {}

    def dfs(u, seen):
        for v in adj[u]:
            if v in seen:
                continue
            seen.add(v)

            if v not in matchR or dfs(matchR[v], seen):
                matchR[v] = u
                return True
        return False

    for u in adj:
        dfs(u, set())

    matchL = {u: v for v, u in matchR.items()}
    return matchL


In [24]:
adj = {
    1:  [2, 5, 7],
    2:  [1, 3],
    3:  [4, 8, 9, 12],
    4:  [5],
    5:  [1, 6, 7, 8],
    6:  [3, 9],
    7:  [2, 4, 6],
    8:  [10, 11],
    9:  [5, 7, 12],
    10: [8]
}



matching = max_matching(adj)
print(len(matching))
print(matching)


10
{7: 2, 2: 1, 3: 4, 4: 5, 6: 3, 5: 6, 1: 7, 8: 10, 10: 8, 9: 12}
